In [ ]:
%pylab

In [ ]:
%matplotlib inline

In [ ]:
import GPy, scipy

## Bayesian Linear Regression 

In [ ]:
def phiGen(p):
    return lambda x: np.array([x**i for i in range(p+1)]).T

p = 3
phi = phiGen(p)

In [ ]:
x = np.linspace(-1,1)

plt.plot(x, phi(x));

plt.axhline(0, color='black')
plt.axvline(0, color='black')

# Compute $Cov(f(x_1), f(x_2))$:

In [ ]:
beta_cov = np.eye(p+1)
f_cov = np.dot(phi(x), np.dot(beta_cov, phi(x).T))

plt.imshow(f_cov)
plt.colorbar()

# Sample $f$:

In [ ]:
f = scipy.stats.multivariate_normal.rvs(np.zeros(50), f_cov, size=10).T

plt.plot(x,f);

In [ ]:
# build a function distribution, with stability in covariance

def fdist(x, phi, beta_cov, sigma = 0.0):
    f_cov = np.dot(phi(x), np.dot(beta_cov, phi(x).T)) + np.eye(x.shape[0])*sigma
    
    jit = 1e-6
    excp = None
    while jit<1e1:
        try:
            return scipy.stats.multivariate_normal(np.zeros(x.shape[0]), f_cov+jit*np.eye(x.shape[0]))
        except np.linalg.LinAlgError as e:
            jit = jit*10
            excp = e
    
    raise excp

# Training model on data

### sample data from prior:

In [ ]:
p = 4
phi = phiGen(p)

xsamp = np.random.choice(x, size=10, replace=False)
xsamp.sort()

fsamp = fdist(xsamp, phiGen(p), np.eye(p+1)).rvs()

plt.scatter(xsamp, fsamp)
plt.axhline(0, color='black')
plt.axvline(0, color='black')

### compute conditional distribution:

$cov(f(x_2), f(x_1))$:

In [ ]:
kzx = np.dot(phi(x), np.dot(np.eye(p+1), phi(xsamp).T))

$cov(f(x_2), f(x_2))$:

In [ ]:
kzz = np.dot(phi(x), np.dot(np.eye(p+1), phi(x).T))

$cov(f(x_1), f(x_1))$

In [ ]:
kxx = np.dot(phi(xsamp), np.dot(np.eye(p+1), phi(xsamp).T)) 

# stabilize matrix
kxx += np.eye(xsamp.shape[0]) * 1e-9

In [ ]:
plt.imshow(kxx)

In [ ]:
plt.imshow(np.linalg.inv(kxx))
plt.colorbar()

conditional:

In [ ]:
mu_pred = np.dot(kzx, np.dot(np.linalg.inv(kxx), fsamp))
cov_pred = kzz - np.dot(kzx, np.dot(np.linalg.inv(kxx), kzx.T))
# cov_pred[cov_pred<1e-9] = 1e-9
std_pred = np.sqrt(np.diag(cov_pred))

plt.plot(x, mu_pred)
plt.fill_between(x, mu_pred-2*std_pred, mu_pred+2*std_pred, alpha=.1)
plt.scatter(xsamp, fsamp)

### Add observation noise:

In [ ]:
x = np.linspace(-1,1)

sigma = 5e-1
p = 4
phi = phiGen(p)
xsamp = np.random.uniform(x.min(), x.max(), size=10)
fsamp = fdist(xsamp, phiGen(p), np.eye(p+1), sigma=sigma).rvs()

plt.scatter(xsamp, fsamp)
plt.axhline(0, color='black')
plt.axvline(0, color='black')

In [ ]:
kzx = np.dot(phiGen(p)(x), np.dot(np.eye(p+1), phiGen(p)(xsamp).T))
kzz = np.dot(phiGen(p)(x), np.dot(np.eye(p+1), phiGen(p)(x).T))
kxx = np.dot(phiGen(p)(xsamp), np.dot(np.eye(p+1), phiGen(p)(xsamp).T)) + sigma*np.eye(xsamp.shape[0])

mu_pred = np.dot(kzx, np.dot(np.linalg.inv(kxx), fsamp))
cov_pred = kzz - np.dot(kzx, np.dot(np.linalg.inv(kxx), kzx.T))
std_pred = np.sqrt(np.diag(cov_pred))

plt.plot(x, mu_pred)
plt.fill_between(x, mu_pred-2*std_pred, mu_pred+2*std_pred, alpha=.1)
plt.scatter(xsamp, fsamp)

In [ ]:
plt.plot(x, mu_pred)
plt.fill_between(x, mu_pred-2*std_pred, mu_pred+2*std_pred, alpha=.1)
plt.plot(x, scipy.stats.multivariate_normal.rvs(mu_pred, cov_pred, size=10).T,alpha=.5);

compute sub-components

In [ ]:
psi = [0] + [1] + [0]*(p-1)
psi*phi(x)

In [ ]:
kzx = np.dot(psi*phi(x), np.dot(np.eye(p+1), phi(xsamp).T))
kzz = np.dot(psi*phi(x), np.dot(np.eye(p+1), (psi*phi(x)).T))
kxx = np.dot(phi(xsamp), np.dot(np.eye(p+1), phi(xsamp).T)) + sigma*np.eye(xsamp.shape[0])

mu_pred = np.dot(kzx, np.dot(np.linalg.inv(kxx), fsamp))
cov_pred = kzz - np.dot(kzx, np.dot(np.linalg.inv(kxx), kzx.T))
# cov_pred[cov_pred<0] = 0
std_pred = np.sqrt(np.diag(cov_pred))

# plt.subplot(121)
plt.plot(x, mu_pred)
plt.fill_between(x, mu_pred-2*std_pred, mu_pred+2*std_pred, alpha=.1)
plt.scatter(xsamp, fsamp)

In [ ]:
for i in range(p+1):
    psi = [0]*(p+1)
    psi[i] = 1
    
    kzx = np.dot(psi*phi(x), np.dot(np.eye(p+1), phi(xsamp).T))
    kzz = np.dot(psi*phi(x), np.dot(np.eye(p+1), (psi*phi(x)).T))
    kxx = np.dot(phi(xsamp), np.dot(np.eye(p+1), phi(xsamp).T)) + sigma*np.eye(xsamp.shape[0])

    mu_pred = np.dot(kzx, np.dot(np.linalg.inv(kxx), fsamp))
    cov_pred = kzz - np.dot(kzx, np.dot(np.linalg.inv(kxx), kzx.T))
    std_pred = np.sqrt(np.diag(cov_pred))

    plt.plot(x, mu_pred, label='$\phi_%d$'%i)
    plt.fill_between(x, mu_pred-2*std_pred, mu_pred+2*std_pred, alpha=.1)
plt.scatter(xsamp, fsamp,c='k',alpha=.5)
plt.legend()

In [ ]:
# parameter inference
ko = np.eye(p+1)
kfo = np.dot(phi(xsamp), ko)
kf = phi(xsamp).dot(ko).dot(phi(xsamp).T) + sigma*np.eye(xsamp.shape[0])

mu_pred = kfo.T.dot(np.linalg.inv(kf)).dot(fsamp)
cov_pred = ko - kfo.T.dot(np.linalg.inv(kf)).dot(kfo)

plt.errorbar(range(p+1), mu_pred, yerr=np.sqrt(cov_pred.diagonal()))

### Compare to linear regression 

In [ ]:
omega_hat = np.linalg.inv(phi(xsamp).T.dot(phi(xsamp))).dot(phi(xsamp).T).dot(fsamp)

plt.errorbar(range(p+1), mu_pred, yerr=np.sqrt(cov_pred.diagonal()))
plt.scatter(range(p+1), omega_hat)

## GP basics

## Radial basis function kernel

In [ ]:
x = np.linspace(-1,1, 101)[:,None]

In [ ]:
kern = GPy.kern.RBF(1, variance=1., lengthscale=1.)

In [ ]:
plt.figure(figsize=(12,6))

c = 10

plt.subplot2grid((c,c*2+2), (0,0), colspan=c, rowspan=c)
plt.plot(x[x.shape[0]/2,0] - x, kern.K(x)[x.shape[0]/2,:])

plt.subplot2grid((c,c*2+2), (0,c+1), colspan=c, rowspan=c)
plt.imshow(kern.K(x))

ax = plt.subplot2grid((c,c*2+2), (0,c*2+1), colspan=1, rowspan=c)
plt.colorbar(cax=ax)

In [ ]:
plt.figure(figsize=(10,6))
for i, ls in enumerate(10**np.arange(-1, 2, 1.)):
    k = GPy.kern.RBF(1, lengthscale=ls)
    s = scipy.stats.multivariate_normal.rvs(np.zeros(x.shape[0]), k.K(x), size=3).T
    
    plt.subplot(121)
    plt.plot(x[x.shape[0]/2,0] - x, k.K(x)[x.shape[0]/2,:], c='C%d'%i, label='$\ell = %.1lf$'%ls)
    
    plt.subplot(122)
    plt.plot(x, s, c='C%d'%i)
    
plt.subplot(121)
plt.xlabel('$ || x_1 - x_2 ||$', fontsize=14)
plt.legend(fontsize=12)

plt.subplot(122)
plt.xlabel('$x$', fontsize=14)

plt.savefig('figures/lengthscale.pdf',bbox_inches='tight')

In [ ]:
plt.figure(figsize=(10,6))
for i, var in enumerate(10**np.arange(-.5, 1, .5)):
    k = GPy.kern.RBF(1, variance=var)
    s = scipy.stats.multivariate_normal.rvs(np.zeros(x.shape[0]), k.K(x), size=3).T
    
    plt.subplot(121)
    plt.plot(x[x.shape[0]/2,0] - x, k.K(x)[x.shape[0]/2,:], c='C%d'%i, label='$\sigma^2 = %.1lf$'%var)
    
    plt.subplot(122)
    plt.plot(x, s, c='C%d'%i)
    
plt.subplot(121)
plt.xlabel('$ || x_1 - x_2 ||$', fontsize=14)
plt.legend(fontsize=12)

plt.subplot(122)
plt.xlabel('$x$', fontsize=14)

plt.savefig('figures/variance.pdf',bbox_inches='tight')